In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [19]:
from train import setup_cfg, print_args
import argparse
from dassl.utils import setup_logger, set_random_seed, collect_env_info
from dassl.engine import build_trainer
import torch

In [28]:
parser = argparse.ArgumentParser()
parser.add_argument('--root', type=str, default='data', help='path to dataset')
parser.add_argument(
    '--output-dir', type=str, default='output', help='output directory'
)
parser.add_argument(
    '--resume',
    type=str,
    default='',
    help='checkpoint directory (from which the training resumes)'
)
parser.add_argument(
    '--seed',
    type=int,
    default=-1,
    help='only positive value enables a fixed seed'
)
parser.add_argument(
    '--source-domains',
    type=str,
    default=['mnist'],
    nargs='+',
    help='source domains for DA/DG'
)
parser.add_argument(
    '--target-domains',
    type=str,
    default=['usps'],
    nargs='+',
    help='target domains for DA/DG'
)
parser.add_argument(
    '--transforms', type=str, nargs='+', help='data augmentation methods'
)
parser.add_argument(
    '--config-file', type=str, default='configs/trainers/digit5.yaml', help='path to config file'
)
parser.add_argument(
    '--dataset-config-file',
    type=str,
    default='configs/datasets/digit5.yaml',
    help='path to config file for dataset setup'
)
parser.add_argument(
    '--trainer', type=str, default='AdaMatch', help='name of trainer'
)
parser.add_argument(
    '--backbone', type=str, default='', help='name of CNN backbone'
)
parser.add_argument('--head', type=str, default='', help='name of head')
parser.add_argument(
    '--eval-only', action='store_true', help='evaluation only'
)
parser.add_argument(
    '--model-dir',
    type=str,
    default='',
    help='load model from this directory for eval-only mode'
)
parser.add_argument(
    '--load-epoch',
    type=int,
    help='load model weights at this epoch for evaluation'
)
parser.add_argument(
    '--no-train', action='store_true', help='do not call trainer.train()'
)
parser.add_argument(
    'opts',
    default=None,
    nargs=argparse.REMAINDER,
    help='modify config options using the command-line'
)
sys.argv = ['-f']
args = parser.parse_args()

In [29]:
args

Namespace(backbone='', config_file='configs/trainers/digit5.yaml', dataset_config_file='configs/datasets/digit5.yaml', eval_only=False, head='', load_epoch=None, model_dir='', no_train=False, opts=[], output_dir='output', resume='', root='data', seed=-1, source_domains=['mnist'], target_domains=['usps'], trainer='AdaMatch', transforms=None)

In [30]:
cfg = setup_cfg(args)
if cfg.SEED >= 0:
    print('Setting fixed seed: {}'.format(cfg.SEED))
    set_random_seed(cfg.SEED)
setup_logger(cfg.OUTPUT_DIR)

if torch.cuda.is_available() and cfg.USE_CUDA:
    torch.backends.cudnn.benchmark = True

print_args(args, cfg)
print('Collecting env info ...')
print('** System info **\n{}\n'.format(collect_env_info()))

trainer = build_trainer(cfg)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/angyizhe/code/SLAB/AdaMatch-PyTorch/data/digit5/mnist/train_images'